In [1]:
# !pip install python-google-places


  Created wheel for python-google-places: filename=python_google_places-1.4.2-py3-none-any.whl size=13608 sha256=a8c786459155824dd82f952848eb3ea74e53bb1b8d88dab364242d9a6898fdee
  Stored in directory: c:\users\mohamed taha\appdata\local\pip\cache\wheels\85\4f\ba\8f83abf0467624401fc6b855afc8da1dbba20d087e6502d557
Successfully built python-google-places


In [3]:
# !pip install geopy

In [27]:
# import torch
# torch.__version__

'1.6.0'

In [1]:
from googleplaces import GooglePlaces, types, lang
import requests
import json
import requests
import urllib.parse
import pandas as pd
import os
import geopy.distance
from geopy import geocoders
from geopy.geocoders import GoogleV3
import requests
from bs4 import BeautifulSoup
import numpy as np

# Use your own API key for making api request calls
API_KEY = """Enter your API Code here"""

# Initialising the GooglePlaces constructor
google_places = GooglePlaces(API_KEY)
######################################################################################
def get_lat_lng(address):

  g = GoogleV3(api_key=API_KEY)
  loc_coordinates = []
  loc_address = []
  try:
      inputAddress = address
      location = g.geocode(inputAddress, timeout=15)
      loc_coordinates.append((location.latitude, location.longitude))
      loc_address.append(inputAddress)
      # print(loc_address)
      # print(loc_coordinates)
      return location.latitude , location.longitude
  except Exception as e:
      print('Error, skipping address...', e)



def distance(lat1 , lng1 , lat2, lng2):
  coords_1 = (lat1, lng1)
  coords_2 = (lat2, lng2)
  try:
      return (geopy.distance.vincenty(coords_1, coords_2).m)
  except:
      return (geopy.distance.geodesic(coords_1, coords_2).m)

######################################################################################################

def search_results(words):
  try:
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36"}
    URL     = "https://www.google.com/search?q=" + words
    result = requests.get(URL, headers=headers) 


    soup = BeautifulSoup(result.content, 'html.parser')


    total_results_text = soup.find("div", {"id": "result-stats"}).find(text=True, recursive=False) # this will give you the outer text which is like 'About 1,410,000,000 results'
    results_num = ''.join([num for num in total_results_text if num.isdigit()]) # now will clean it up and remove all the characters that are not a number .
    return results_num * 100 + np.random.randint(0,9)
  except:
    return  0 + np.random.randint(0,99)

In [13]:
University = 'جامعة عين شمس'
Company = 'ITI Smart Village'
BirthPlace = 'قويسنا المنوفية'
LiveIn = '82 شارع عباس العقاد مدينة نصر'

num_of_places = 1
def  construct_graph(University,Company, BirthPlace, LiveIn , USER_NUMBER ):
  personal_info = [University,Company,BirthPlace,LiveIn]
  # Iterate over the search results
  types_dict = {"Bank":[types.TYPE_BANK , 500] , "bus stop":[types.TYPE_BUS_STATION , 1000], "Convenience store" :[types.TYPE_CONVENIENCE_STORE,400],
                "supermarket": [types.TYPE_GROCERY_OR_SUPERMARKET,100], "hospital":[types.TYPE_HOSPITAL,500],
                "Dinning room": [types.TYPE_RESTAURANT,200],"Cinema":[types.TYPE_MOVIE_THEATER,1000]}

  ##################### PATH USER ##################################################
  User_list = [
                ["User", "Worked at" , Company ],
                ["User", "Graduated from" , University],
                ["User", "live", LiveIn ],
                ["User", "born in", BirthPlace]
                ]

  ##################### PATH PersonEntity  ####################################################
  path_person_entity_dict = {}
  places_dict= {}

  for info in personal_info:
    PersonEntity_list_S = []
    lat,lng = get_lat_lng( info )
    #new
    for place_type in types_dict.keys():
        query_result = google_places.nearby_search(lat_lng ={'lat': lat, 'lng': lng}, radius = types_dict[place_type][1], types =[types_dict[place_type][0]])
        extra_radius = 0
        while len(query_result.places) < num_of_places:
            extra_radius += 300
            #increase radius
            query_result = google_places.nearby_search(lat_lng ={'lat': lat, 'lng': lng}, radius = types_dict[place_type][1] + extra_radius, types =[types_dict[place_type][0]])
            if extra_radius >= 500:
              break

        i = 0
        for place in query_result.places:
          i+=1
          PersonEntity_list_S.append([info , place_type , place.name]) #return the list
          #new
          places_dict[place.name] =  [ place_type , float(place.geo_location['lat']) , float(place.geo_location['lng']) ]


          if i >= num_of_places:
            break

    path_person_entity_dict[info] = PersonEntity_list_S

  ###################### PATH NewAdd ###############################################
  new_add = []
  i = 1 
  for original_place in places_dict.keys():
    lat1,lng1 = places_dict[original_place][1] , places_dict[original_place][2]
    for second_place in places_dict.keys():

      lat2 , lng2 = places_dict[second_place][1] , places_dict[second_place][2]
      the_distance = distance(lat1 , lng1 , lat2, lng2)
      place_type = places_dict[second_place][0]
      if (the_distance < 250)  and (second_place != original_place) :
            

            # print(the_distance,[original_place , place_type , second_place ])
            new_add.append( [original_place , place_type , second_place ] )
            

  ########################## entity_description  PersonEntity ########################################################

  person_entity_dict = {}
  for info in personal_info:
    if info == Company: place_type = 'company'
    elif info == University: place_type = 'University'
    elif info == BirthPlace: place_type = 'Residential area'
    elif info == LiveIn: place_type = 'Residential area'
    person_entity_dict[info] = { "id": str(hash(info))[1:9] , "loc":  list( str(i) for i in get_lat_lng(info)) , "type": place_type } 

  ############################ entity_description  OneStepEntity #########################################################################################################
  one_step_entity_dict = {}
  for one_step_entity in places_dict.keys():
    the_type , lat , lng = places_dict[one_step_entity][0] , places_dict[one_step_entity][1] , places_dict[one_step_entity][2]

    one_step_entity_dict[one_step_entity] = { "id": str(hash(one_step_entity))[1:9] , "loc":  [ lat , lng ] , "type": the_type } 

                

  ###############################################################################################
  graph = {} # Done
  graph['paths'] = {} #Done
  graph['paths']['User'] = User_list #Done
  graph['paths']['PersonEntity'] = path_person_entity_dict #Done
  graph['paths']['NewAdd'] = new_add #Done
  graph['entity_description'] = {} #Done
  graph['entity_description']['User'] = USER_NUMBER #Done
  graph['entity_description']['PersonEntity'] = person_entity_dict #Done
  graph['entity_description']['OneStepEntity'] = one_step_entity_dict #Done

  ######################################################################################################################################################
  rls = []


  for triples in graph["paths"]["PersonEntity"].values():
      for (h, r, t) in triples:
          rls.append("“{}” “{}”".format(str(h), str(t)))
  for (h, r, t) in graph["paths"]["NewAdd"]:
      if "“{}” “{}”".format(str(h), str(t)) not in rls and "“{}” “{}”".format(str(t), str(h)) not in rls:
          rls.append("“{}” “{}”".format(str(h), str(t)))


  se_result = {k: str(search_results(k)) for k in rls }

  return graph , se_result



In [14]:
graph , se_result = construct_graph(University,Company, BirthPlace, LiveIn , 907 )
graph

{'paths': {'User': [['User', 'Worked at', 'ITI Smart Village'],
   ['User', 'Graduated from', 'جامعة عين شمس'],
   ['User', 'live', '82 شارع عباس العقاد مدينة نصر'],
   ['User', 'born in', 'قويسنا المنوفية']],
  'PersonEntity': {'جامعة عين شمس': [['جامعة عين شمس',
     'Bank',
     'Agricultural Development Bank and credit-'],
    ['جامعة عين شمس', 'hospital', 'مركز حبيب الطبي HABIB MEDICAL CENTER'],
    ['جامعة عين شمس', 'Dinning room', 'ZAD restaurant and cafe']],
   'ITI Smart Village': [['ITI Smart Village', 'Bank', 'Credit Agricole Bank'],
    ['ITI Smart Village', 'Convenience store', 'Circlek Egypt'],
    ['ITI Smart Village', 'hospital', 'Go Clinic'],
    ['ITI Smart Village', 'Dinning room', 'الشبراوي'],
    ['ITI Smart Village',
     'Cinema',
     'Renaissance Cinema Dandy Mall - رينيسانس سينما داندي مول']],
   'قويسنا المنوفية': [['قويسنا المنوفية',
     'supermarket',
     'El Qods Supermarket'],
    ['قويسنا المنوفية', 'hospital', 'health unit'],
    ['قويسنا المنوفية', '

In [15]:
se_result

se_result = {k: str(search_results(k)) for k in se_result.keys() }
se_result

{'“جامعة عين شمس” “Agricultural Development Bank and credit-”': '95',
 '“جامعة عين شمس” “مركز حبيب الطبي HABIB MEDICAL CENTER”': '56',
 '“جامعة عين شمس” “ZAD restaurant and cafe”': '81',
 '“ITI Smart Village” “Credit Agricole Bank”': '42',
 '“ITI Smart Village” “Circlek Egypt”': '6',
 '“ITI Smart Village” “Go Clinic”': '70',
 '“ITI Smart Village” “الشبراوي”': '12',
 '“ITI Smart Village” “Renaissance Cinema Dandy Mall - رينيسانس سينما داندي مول”': '28',
 '“قويسنا المنوفية” “El Qods Supermarket”': '95',
 '“قويسنا المنوفية” “health unit”': '72',
 '“قويسنا المنوفية” “El Salakawy Sons”': '37',
 '“82 شارع عباس العقاد مدينة نصر” “CIB - Commercial International Bank”': '77',
 '“82 شارع عباس العقاد مدينة نصر” “Al Andals Hospital”': '83',
 '“82 شارع عباس العقاد مدينة نصر” “الفردوس”': '52',
 '“82 شارع عباس العقاد مدينة نصر” “سوبر ماركت”': '33',
 '“82 شارع عباس العقاد مدينة نصر” “مستشفى المصيلحى”': '93',
 '“82 شارع عباس العقاد مدينة نصر” “Dough & Dough”': '51',
 '“82 شارع عباس العقاد مدينة نصر” “D

In [24]:
# new_add = []
# i = 1 
# for original_place in places_dict.keys():
#   lat1,lng1 = places_dict[original_place][1] , places_dict[original_place][2]
#   for second_place in places_dict.keys():

#     lat2 , lng2 = places_dict[second_place][1] , places_dict[second_place][2]
#     the_distance = distance(lat1 , lng1 , lat2, lng2)
#     place_type = places_dict[second_place][0]
#     if (the_distance < 500)  and (second_place != original_place) :
          

#           print(the_distance,[original_place , place_type , second_place ])
#           new_add.append( [original_place , place_type , second_place ] )
# new_add

In [32]:
# places_dict

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
with open(os.path.join(r"C:\Users\Mohamed Taha\Desktop\FinalWork", "test1" + ".json"), 'w' , encoding= 'utf8') as outfile:
         json.dump(graph, outfile, indent=4)

In [17]:
with open(os.path.join(r"C:\Users\Mohamed Taha\Desktop\FinalWork", "se_result1" + ".json"), 'w' , encoding= 'utf8') as outfile:
         json.dump(se_result, outfile, indent=4)

In [ ]:
with open(os.path.join("/content/drive/MyDrive/DialogueBased", "test1" + ".json"),  encoding= 'utf8') as f:
        result = json.load(f)
result

{'entity_description': {'OneStepEntity': {'Ahmed Zewail Park': {'id': '45618764',
    'loc': [31.1140373, 30.9372549],
    'type': 'garden'},
   'BANK MISR': {'id': '88014408',
    'loc': [29.9921701, 32.4946432],
    'type': 'Bank'},
   'Banque Misr': {'id': '29086067',
    'loc': [31.11250620000001, 30.93873520000001],
    'type': 'Bank'},
   'Cinema Galaxy': {'id': '67973293',
    'loc': [31.0373931, 31.3613217],
    'type': 'Cinema'},
   'El shamly home': {'id': '54180176',
    'loc': [31.11313479999999, 30.94645130000001],
    'type': 'garden'},
   'Elaf Egypt': {'id': '71245948',
    'loc': [31.03707589999999, 31.3563145],
    'type': 'hospital'},
   'Elite Child Health Center': {'id': '58627221',
    'loc': [31.1122559, 30.9497577],
    'type': 'hospital'},
   'Miramar Cafe': {'id': '55211086',
    'loc': [29.9986494, 32.4988005],
    'type': 'Dinning room'},
   'Zombie VR': {'id': '22648553',
    'loc': [31.1145199, 30.9430237],
    'type': 'Cinema'},
   'أسواق الشيتانى': {'id'

In [68]:


# graph['paths']['NewAdd'] = new_add  #Done
# path_person_entity_dict = graph['paths']['PersonEntity'] 



# rls = []


# for triples in graph["paths"]["PersonEntity"].values():
#     for (h, r, t) in triples:
#         rls.append("“{}” “{}”".format(str(h), str(t)))
# for (h, r, t) in graph["paths"]["NewAdd"]:
#     if "“{}” “{}”".format(str(h), str(t)) not in rls and "“{}” “{}”".format(str(t), str(h)) not in rls:
#         rls.append("“{}” “{}”".format(str(h), str(t)))


# data = {k: search_results(k) for k in rls }

# # with open(os.path.join(DATA_RESULT, str(i) + ".json"), 'w') as outfile:
# #     json.dump(data, outfile, indent=4)

# print(data)


{'“جامعة السويس” “BANK MISR”': 81, '“جامعة السويس” “مكتب تموين جمعيتي”': 33, '“جامعة السويس” “مركز براعم للاطفال”': 56, '“جامعة السويس” “Miramar Cafe”': 40, '“Information Technology Institute Mansoura Branch” “فر ع جامعة المنصورة”': 1, '“Information Technology Institute Mansoura Branch” “منصة مكوك للمتاجر الالكترونية”': 57, '“Information Technology Institute Mansoura Branch” “كارفور ماركت”': 38, '“Information Technology Institute Mansoura Branch” “Elaf Egypt”': 19, '“Information Technology Institute Mansoura Branch” “كافتيريا ومطعم نيولوك &”': 15, '“Information Technology Institute Mansoura Branch” “Cinema Galaxy”': 35, '“مدينة كفرالشيخ” “Banque Misr”': 9, '“مدينة كفرالشيخ” “سوبر ماركت الربيع”': 55, '“مدينة كفرالشيخ” “أسواق الشيتانى”': 84, '“مدينة كفرالشيخ” “مستشفى صندلا المركزى.. Central Hospital of Sundela”': 96, '“مدينة كفرالشيخ” “ابوعبده”': 21, '“مدينة كفرالشيخ” “دار عرض سينما الجمهورية”': 67, '“شارع الجزائر تقسيم2 كفرالشيخ” “بنك الاتحاد الوطني”': 57, '“شارع الجزائر تقسيم2 كفرالشيخ

In [69]:
for i in data.keys():
  print(i , data[i] )

“جامعة السويس” “BANK MISR” 81
“جامعة السويس” “مكتب تموين جمعيتي” 33
“جامعة السويس” “مركز براعم للاطفال” 56
“جامعة السويس” “Miramar Cafe” 40
“Information Technology Institute Mansoura Branch” “فر ع جامعة المنصورة” 1
“Information Technology Institute Mansoura Branch” “منصة مكوك للمتاجر الالكترونية” 57
“Information Technology Institute Mansoura Branch” “كارفور ماركت” 38
“Information Technology Institute Mansoura Branch” “Elaf Egypt” 19
“Information Technology Institute Mansoura Branch” “كافتيريا ومطعم نيولوك &” 15
“Information Technology Institute Mansoura Branch” “Cinema Galaxy” 35
“مدينة كفرالشيخ” “Banque Misr” 9
“مدينة كفرالشيخ” “سوبر ماركت الربيع” 55
“مدينة كفرالشيخ” “أسواق الشيتانى” 84
“مدينة كفرالشيخ” “مستشفى صندلا المركزى.. Central Hospital of Sundela” 96
“مدينة كفرالشيخ” “ابوعبده” 21
“مدينة كفرالشيخ” “دار عرض سينما الجمهورية” 67
“شارع الجزائر تقسيم2 كفرالشيخ” “بنك الاتحاد الوطني” 57
“شارع الجزائر تقسيم2 كفرالشيخ” “سليم ماركت” 6
“شارع الجزائر تقسيم2 كفرالشيخ” “اسواق عباد الرحمن” 31

In [52]:
graph["paths"]["NewAdd"]
# distance(31.04168 , 31.35636 , 31.0382198, 31.35492369999999)

[['مكتب تموين جمعيتي', 'Dinning room', 'Miramar Cafe'],
 ['Miramar Cafe', 'supermarket', 'مكتب تموين جمعيتي'],
 ['فر ع جامعة المنصورة', 'supermarket', 'كارفور ماركت'],
 ['فر ع جامعة المنصورة', 'Dinning room', 'كافتيريا ومطعم نيولوك &'],
 ['منصة مكوك للمتاجر الالكترونية', 'Dinning room', 'كافتيريا ومطعم نيولوك &'],
 ['كارفور ماركت', 'Bank', 'فر ع جامعة المنصورة'],
 ['كارفور ماركت', 'hospital', 'Elaf Egypt'],
 ['كارفور ماركت', 'Dinning room', 'كافتيريا ومطعم نيولوك &'],
 ['Elaf Egypt', 'supermarket', 'كارفور ماركت'],
 ['Elaf Egypt', 'Cinema', 'Cinema Galaxy'],
 ['كافتيريا ومطعم نيولوك &', 'Bank', 'فر ع جامعة المنصورة'],
 ['كافتيريا ومطعم نيولوك &',
  'Convenience store',
  'منصة مكوك للمتاجر الالكترونية'],
 ['كافتيريا ومطعم نيولوك &', 'supermarket', 'كارفور ماركت'],
 ['Cinema Galaxy', 'hospital', 'Elaf Egypt'],
 ['Banque Misr', 'supermarket', 'أسواق الشيتانى'],
 ['Banque Misr',
  'hospital',
  'مستشفى صندلا المركزى.. Central Hospital of Sundela'],
 ['Banque Misr', 'Dinning room', 'ابوعبد

In [54]:
import numpy as np 

In [63]:
np.random.randint(0,5)

2